# Generate data using api (for now, DR dataset only)

In [25]:
df = pd.read_csv(r".\data\released_data_mentalllama\train_data\complete_data\DR\reddit_valid.csv")

df.question.unique()

array([' Question: The answer to the question "Does the poster suffers from depression?" is '],
      dtype=object)

#### Sampling without replacement from original data

In [ ]:
import random
from update_prompt import update_prompt_with_csv


prompt_path = r".\prompts\prompt_complete_DR.txt"
csv_valid_path = r".\data\released_data_mentalllama\train_data\complete_data\DR\reddit_valid.csv"
csv_train_path = r".\data\released_data_mentalllama\train_data\complete_data\DR\reddit_train.csv"

random.seed(42) # For reproducibility
n = 25

update_prompt_with_csv(prompt_path, csv_valid_path, csv_train_path, n)

Updated '.\prompts\prompt_complete_DR.txt' with 25 new example rows.


#### data generation

In [8]:
from openai import OpenAI

client_ds = OpenAI(api_key="sk-ed8f7a41078845a5bf6d6db82686ca08", base_url="https://api.deepseek.com")

In [37]:
import csv
import io
import os

with open('./prompts/prompt_complete_DR.txt', 'r') as file:
    content = file.read()


response = client_ds.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a mental health data generation assistant"},
        {"role": "user", "content": content},
    ],
    stream=False,
    temperature=1.5,
    max_tokens=8192
)

print(response.choices[0].message.content)


reader = csv.reader(io.StringIO(response.choices[0].message.content))
rows = list(reader)

header = ['post', 'question', 'response']
output_csv = './data/data_generated/DR_generated.csv'

file_exists = os.path.exists(output_csv)

with open(output_csv, 'a' if file_exists else 'w', newline='', encoding='utf-8') as out_f:
    writer = csv.writer(out_f)
    if not file_exists or os.stat(output_csv).st_size == 0:
        writer.writerow(header)
    writer.writerows(rows)

print(f"CSV file '{output_csv}' updated with {len(rows)} new rows of data.")

"Post: I feel like I'm drowning in my own thoughts. Every day is a battle to get out of bed, and even when I do, I feel like I'm just going through the motions. I used to love reading, but now I can't even focus on a single page. My friends keep asking me to hang out, but I just can't bring myself to go. I feel like I'm letting everyone down, including myself. I don't know why I feel this way, but it's like a heavy weight is constantly pressing down on my chest. I try to smile and act normal, but inside, I feel so empty. I don't know how much longer I can keep pretending everything is okay.","Question: The answer to the question "Does the poster suffers from depression?" is","yes. Reasoning: The post exhibits multiple symptoms of depression, including persistent sadness, lack of interest in previously enjoyed activities, difficulty concentrating, and feelings of worthlessness. The poster describes a constant emotional struggle, feeling empty and overwhelmed by negative thoughts. The me

In [38]:
# data prepossessing
pd.read_csv('./data/data_generated/DR_generated.csv')

,post,question,response
0,Post: I feel like I’m drowning in my own thoug...,Question: The answer to the question 'Does the...,yes. Reasoning: The post exhibits strong emoti...
1,Post: I’ve been feeling so off lately. I can’t...,Question: The answer to the question 'Does the...,no. Reasoning: While the post describes signif...
